In [32]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
from scipy.ndimage import gaussian_filter1d
import plotly.graph_objects as go


def make_agora_plot(filename: str):
    df = pd.read_csv(filename)
    df["Wind (total)"] = df["Wind offshore"] + df["Wind onshore"]

    df_melted = pd.melt(df, id_vars=["date_id"], value_vars=["Biomass", "Hydro", "Wind (total)", "Solar", "Conventional"])

    fig = px.area(df_melted, x="date_id", y="value", color="variable", line_group="variable",
        color_discrete_map={
            "Biomass": "green",
            "Hydro": "blue",
            "Solar": "yellow",
            "Conventional": "grey",
            "Wind": "lightgrey"
    })

    return fig

weekly_fig = make_agora_plot("power_generation_and_consumption_week_july.csv")

# Also add the carbon emissions from elecricitymaps
carbon_df = pd.read_csv("co₂_emissions_from_power_generation.csv")

combined_weekly_fig = make_subplots(specs=[[{"secondary_y": True}]])

combined_weekly_fig.add_trace(
    go.Scatter(x=carbon_df['date_id'], y=carbon_df['Grid emission factor'], line={'color':'red'}, showlegend=False),
    secondary_y=True
)

for trace in weekly_fig.data:
    combined_weekly_fig.add_trace(
    trace,
    secondary_y=False
)
    
combined_weekly_fig.update_layout(
    yaxis=dict(
        title="GWh/hour",
    ),
    yaxis2=dict(
        title="Average carbon emissions in gCO2/kWh",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        )
    ),
)

combined_weekly_fig.update_layout(legend_orientation="h")

combined_weekly_fig.update_xaxes(
    tickformat="%b %d"  # Set your desired format
)

combined_weekly_fig.show()

yearly_fig = make_agora_plot("power_generation_and_consumption_year.csv")

# Also add a smoothed renewable share to the yearly plot

combined_yearly_fig = make_subplots(specs=[[{"secondary_y": True}]])


for trace in yearly_fig.data:
    combined_yearly_fig.add_trace(
    trace,
    secondary_y=False
)
    
yearly_df = pd.read_csv("power_generation_and_consumption_year.csv")
yearly_df["Renewable share smoothed"] = gaussian_filter1d(yearly_df['Renewable share'], sigma=4)


renewables_share_fig = go.Scatter(x=yearly_df["date_id"], y=yearly_df["Renewable share smoothed"], 
                                  line={'color':'red'}, showlegend=False
)


combined_yearly_fig.add_trace(
    renewables_share_fig,
    secondary_y=True
)

combined_yearly_fig.update_layout(
    yaxis=dict(
        title="GWh/hour",
    ),
    yaxis2=dict(
        title="Share of renewables in %",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        )
    ),
)

combined_yearly_fig.update_layout(legend_orientation="h")

combined_yearly_fig.show()


pio.write_image(combined_weekly_fig, file="energy_production_week.pdf", format='pdf')
pio.write_image(combined_yearly_fig, file="energy_production_year.pdf", format='pdf')